###Baseline Model

In [ ]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing

In [ ]:
def ts_split(series, test_size=0.2):
    split = int(len(series) * (1 - test_size))
    return series[:split], series[split:]

In [ ]:
all_results = []

group_cols = ['Category', 'Region', 'Store ID', 'Product ID']

In [ ]:
#prophet uni
for (cat, reg, store, prod), grp in df.groupby(group_cols):

    ts = grp.sort_values('Date').set_index('Date')['Demand']

    if len(ts) < 40:
        continue

    train, test = ts_split(ts)

        # ----------------------------
    # Prophet - Univariate
    # ----------------------------
    try:
        prophet_train = train.reset_index().rename(
            columns={'Date': 'ds', 'Demand': 'y'}
        )

        prophet_model = Prophet()
        prophet_model.fit(prophet_train)

        future = prophet_model.make_future_dataframe(
            periods=len(test),
            freq='D'
        )

        forecast = prophet_model.predict(future)
        prophet_forecast = forecast.tail(len(test))['yhat'].values

        all_results.append({
            'Model': 'Prophet Univariate',
            'Category': cat,
            'Region': reg,
            'Store ID': store,
            'Product ID': prod,
            'MAPE': mean_absolute_percentage_error(test, prophet_forecast),
            'R2': r2_score(test, prophet_forecast)
        })
    except Exception as e:
        pass

###Fine-tuned Model

In [ ]:
df['Demand_log'] = np.log1p(df['Demand'])

def cap_outliers(series, q=0.99):
    cap = series.quantile(q)
    return np.where(series > cap, cap, series)

df['Demand_log'] = (
    df.groupby(['Category','Region','Store ID','Product ID'])['Demand_log']
      .transform(cap_outliers)
)

In [ ]:
for lag in [1, 7, 14]:
    df[f'lag_{lag}'] = (
        df.groupby(['Category','Region','Store ID','Product ID'])['Demand_log']
          .shift(lag)
    )

In [ ]:
df['day'] = df['Date'].dt.day
df['month'] = df['Date'].dt.month
df['dayofweek'] = df['Date'].dt.dayofweek

In [ ]:
def train_test_split_ts(df, test_ratio=0.2):
    split = int(len(df) * (1 - test_ratio))
    return df.iloc[:split], df.iloc[split:]

In [ ]:
#prophet uni
# No longer need to initialize prophet_results as a separate list

for (cat, reg, store, prod), grp in df.groupby(
    ['Category','Region','Store ID','Product ID']
):

    ts = grp[['Date','Demand_log']].dropna()

    if len(ts) < 40:
        continue

    train, test = train_test_split_ts(ts)

    train_df = train.rename(columns={'Date':'ds','Demand_log':'y'})
    test_df = test[['Date']].rename(columns={'Date':'ds'})

    best_mape = np.inf

    for cps in [0.01, 0.05, 0.1, 0.3]:
        p = Prophet(
            weekly_seasonality=True,
            yearly_seasonality=False,
            changepoint_prior_scale=cps
        )
        p.fit(train_df)

        pred = np.expm1(p.predict(test_df)['yhat'])
        actual = np.expm1(test['Demand_log'])

        mape = mean_absolute_percentage_error(actual, pred)

        if mape < best_mape:
            best_mape = mape
            best_model = (p, mape, r2_score(actual, pred))

    all_results.append({
        'Model': 'Prophet (Fine-Tuned)',
        'Category': cat,
        'Region': reg,
        'Store ID': store,
        'Product ID': prod,
        'MAPE': best_model[1],
        'R2': best_model[2]
    })